# Hypothesentests 1

### Daten aus der CSV-Datei lesen

Nun benutzen wir pandas um die [CSV-Datei](https://de.wikipedia.org/wiki/CSV_(Dateiformat)) einzulesen. Damit das funktioniert muss vorher noch das pandas-Paket mit dem `import`-Befehl unter dem Namen `pd` verfügbar gemacht werden.

Dabei wird ein sogenannter [DataFrame](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html) mit dem Namen `nba` erzeugt. Vereinfacht kann man sich darunter eine Tabelle mit Spalten und Zeilen vorstellen. DataFrames sind **die** zentralen Datentypen in pandas.   

Mit dem Befehl `type()` kann der Datentyp eines jeden Python-Objekts ausgegeben werden. 

In [196]:
import pandas as pd
import numpy as np
nba = pd.read_csv("nba_all_elo.csv")
type(nba)
# Expected:
# <class 'pandas.core.frame.DataFrame'>

pandas.core.frame.DataFrame

In [197]:
nba.head()

,gameorder,game_id,lg_id,_iscopy,year_id,date_game,seasongame,is_playoffs,team_id,fran_id,...,win_equiv,opp_id,opp_fran,opp_pts,opp_elo_i,opp_elo_n,game_location,game_result,forecast,notes
0,1,194611010TRH,NBA,0,1947,11/1/1946,1,0,TRH,Huskies,...,40.294830,NYK,Knicks,68,1300.0000,1306.7233,H,L,0.640065,NaN
1,1,194611010TRH,NBA,1,1947,11/1/1946,1,0,NYK,Knicks,...,41.705170,TRH,Huskies,66,1300.0000,1293.2767,A,W,0.359935,NaN
2,2,194611020CHS,NBA,0,1947,11/2/1946,1,0,CHS,Stags,...,42.012257,NYK,Knicks,47,1306.7233,1297.0712,H,W,0.631101,NaN
3,2,194611020CHS,NBA,1,1947,11/2/1946,2,0,NYK,Knicks,...,40.692783,CHS,Stags,63,1300.0000,1309.6521,A,L,0.368899,NaN
4,3,194611020DTF,NBA,0,1947,11/2/1946,1,0,DTF,Falcons,...,38.864048,WSC,Capitols,50,1300.0000,1320.3811,H,L,0.640065,NaN


# Aufgabe: Finals
![finals](./finals.png)

Seit 2005 werden die Playoffs, das Finalturnier der NBA, in einem Wettbewerb im Best-of-Seven Moduls gespielt.
Eine Finalserie hat also zwischen 4 und 7 Spielen zwischen einer Spielpaarung. Dabei werden die ersten vier Spiele je 2 Spiele am Heimort der ersten Mannschaft und anschließend zwei Spiele am Heimort der anderen Mannschaft gespielt. Danach wird der Spielort pro Spiel gewechselt. Folglich startet eine Mannschaft mit zwei Auswärtsspielen in eines solche Finalserie. Siehe: https://en.wikipedia.org/wiki/NBA_playoffs

![finals](./finals_PHX_DAL.png)



Da im Sport Erfahrungen, Binsenweisheiten und andere Mutmaßungen sehr weit verbreitet ist können man sich folgenden Ausspruch vorstellen:

>"Das Auswärtsteam wird eine Serie eher gewinnen wenn Sie das erste Auswärtsspiel (Spiel 1 der Serie) gewinnt und das zweite verliert als anders herum."

Da wir die Daten zu allen Spielen haben lässt sich das ja nun eigentlich sehr einfach nachprüfen.

In den nächsten beiden Einheiten werden wird die Daten soweit aufbereiten um diese Hypothese zu testen. Dazu müssen wir nun erst einmal die Serien in den Daten nachbilden.


### 1. letzte Vorbereitungsschritte...

#### Ergebnisse für alle ersten und zweiten Spiele einer Serie finden und in eine neue Spalte `game1_2` eintragen. 

Das Format ist `W_L`, `W_W`, `L_W` bzw. `L_L`. Die Ergebnisse sollten ab dieser Stelle sinnvollerweise auf Kopien gefiltert werden. 
Außerdem wird nun nur noch eine Zeile pro Serie benötigt. 

Der DataFrame soll zum Schluß die folgende Spalten haben: 
`year_id`, `date_game` (= erstes Spiel), `team_id`, `fran_id`, `opp_id`, `opp_fran`, `game_location`, `num_series_wins`, `opp_series_wins`, `series_winner` und `game1_2`.

Um die Ergebnisse der Spiele 1 und 2 zusammen zu bringen müssen in diesem Fall Werte aus zwei verschiedene Zeilen gelesen werden. Dazu könnten Sie zum Beispiel die `transform()` Funktion in verbindung mit einer lambda-Funktion verbinden, die die Zeichenketten verbindet. Siehe: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.transform.html?highlight=transform#pandas.DataFrame.transform 

Eine weitere Möglichkeit wäre die Verwendung von `merge()`, um denn Data Frame mit per cross-join "sich selbst" zu verbinden, und anschließend alle überflüssigen Zeilen herauszufiltern. 

Schritt 1:

Series | year_id | num_series_x | ... | game_result_x | num_series_y | game_result_y 
---|---|---|---|---|---|---
HOU_DAL | 2005 | 1.0 | ... | W | 1.0 | W 
HOU_DAL | 2005 | 2.0 | ... | L | 1.0 | W 
HOU_DAL | 2005 | 1.0 | ... | W | 2.0 | L 
HOU_DAL | 2005 | 2.0 | ... | L | 2.0 | L 

Schritt 2:

Series | year_id | num_series_x | ... | game_result_x | num_series_y | game_result_y 
---|---|---|---|---|---|---
HOU_DAL | 2005 | 1.0 | ... | W | 2.0 | L 

Schritt 3:

Series | year_id |  ... | game1_2
---|---|---|---
HOU_DAL | 2005 |  ... | W_L 



In [198]:
playoffs2005_merged = pd.read_csv("nba_hypo2005.csv")

Kopien löschen per Filter:

In [199]:
playoffs2005_filtered = playoffs2005_merged[playoffs2005_merged["_iscopy"] == 0]

Serien auf das erste Spiel einer Serie verdichten. Achtung: `date_game` und `game_location` beziehen sich nun auf das erste Spiel.

In [200]:
series = pd.DataFrame(playoffs2005_filtered.groupby(["series", "year_id", "team_id", "fran_id", "opp_id", "opp_fran", "game_location", "num_series_wins", "opp_series_wins", "series_winner"]).date_game.min())

Index zurücksetzen und auf `series` und `year_id` neu festlegen.

In [201]:
series.reset_index(inplace=True)

In [202]:
series.set_index(["series","year_id"], inplace=True)

Ergebnisse überprüfen:

In [203]:
series.tail()

team_id  fran_id opp_id   opp_fran game_location  \
series  year_id                                                    
WAS_CHI 2005        WAS  Wizards    CHI      Bulls             H   
        2014        WAS  Wizards    CHI      Bulls             H   
WAS_CLE 2006        WAS  Wizards    CLE  Cavaliers             H   
        2008        WAS  Wizards    CLE  Cavaliers             H   
WAS_IND 2014        WAS  Wizards    IND     Pacers             H   

                 num_series_wins  opp_series_wins series_winner   date_game  
series  year_id                                                              
WAS_CHI 2005                   4                2           WAS  2005-04-30  
        2014                   4                1           WAS  2014-04-25  
WAS_CLE 2006                   2                4           CLE  2006-04-28  
        2008                   2                4           CLE  2008-04-24  
WAS_IND 2014                   2                4           IND  2014-05-09

Für die Spiel 1 + 2 Bewertung einen weiteren Dataframe auf Basis der einzelnen Spiele erstellen und nur die notwendigen Spalten extrahieren. 

In [204]:
playoffs2005_filtered2 =  playoffs2005_filtered.loc[(playoffs2005_filtered["series_game"] == 1.0) | (playoffs2005_filtered["series_game"] == 2.0) ,["year_id","series","series_game","game_result"]]

In [205]:
playoffs2005_filtered2.tail(20)

,year_id,series,series_game,game_result
1513,2015,HOU_DAL,1.0,W
1514,2015,HOU_DAL,2.0,W
1518,2015,ATL_BRK,1.0,W
1519,2015,ATL_BRK,2.0,W
1530,2015,LAC_SAS,1.0,W
1531,2015,LAC_SAS,2.0,L
1544,2015,MEM_POR,1.0,W
1545,2015,MEM_POR,2.0,W
1554,2015,ATL_WAS,1.0,L
1555,2015,ATL_WAS,2.0,W


Die neue Spalte `game1_2` wird per `transform()` und einer `lambda` Funktion erzeugt. Das bedeutet: Es wird für alle Ergebnisse des `groupby()` die Funktion ausgeführt, die im Argument der `transform()`angegeben wurde. Im Vorleigenden Fall gibt es z.B. ein "W" und ein "L" in der Spalte `game_result`. Diese werden dann mit einem "_" verkettet.

In [206]:
playoffs2005_filtered2["game_1_2"] = playoffs2005_filtered2.groupby(["year_id","series"])["game_result"].transform(lambda x : '_'.join(x))

In [207]:
playoffs2005_filtered2.tail(20)

,year_id,series,series_game,game_result,game_1_2
1513,2015,HOU_DAL,1.0,W,W_W
1514,2015,HOU_DAL,2.0,W,W_W
1518,2015,ATL_BRK,1.0,W,W_W
1519,2015,ATL_BRK,2.0,W,W_W
1530,2015,LAC_SAS,1.0,W,W_L
1531,2015,LAC_SAS,2.0,L,W_L
1544,2015,MEM_POR,1.0,W,W_W
1545,2015,MEM_POR,2.0,W,W_W
1554,2015,ATL_WAS,1.0,L,L_W
1555,2015,ATL_WAS,2.0,W,L_W


Auf eine Zeile pro Serie und Paarung (Spiel 1) beschränken.

In [208]:
playoffs2005_filtered2 = playoffs2005_filtered2.loc[(playoffs2005_filtered2["series_game"]==1.0),["year_id", "series", "game_1_2"]]

In [209]:
playoffs2005_filtered2.tail()

,year_id,series,game_1_2
1572,2015,GSW_MEM,W_L
1584,2015,CLE_CHI,L_W
1590,2015,HOU_LAC,L_W
1604,2015,GSW_HOU,W_W
1620,2015,GSW_CLE,W_L


Zu einem Endresultat mit `merge()` zusammenfügen:

In [210]:
result = pd.merge(left=series,right=playoffs2005_filtered2,on=["year_id","series"])

In [211]:
result.tail(20)

,year_id,series,team_id,fran_id,opp_id,opp_fran,game_location,num_series_wins,opp_series_wins,series_winner,date_game,game_1_2
117,2006,SAS_DAL,SAS,Spurs,DAL,Mavericks,H,3,4,DAL,2006-05-07,W_L
118,2009,SAS_DAL,SAS,Spurs,DAL,Mavericks,H,1,4,DAL,2009-04-18,L_W
119,2014,SAS_DAL,SAS,Spurs,DAL,Mavericks,H,4,3,SAS,2014-04-20,W_L
120,2005,SAS_DEN,SAS,Spurs,DEN,Nuggets,H,4,1,SAS,2005-04-24,L_W
121,2007,SAS_DEN,SAS,Spurs,DEN,Nuggets,H,4,1,SAS,2007-04-22,L_W
122,2005,SAS_DET,SAS,Spurs,DET,Pistons,H,4,3,SAS,2005-06-09,W_W
123,2013,SAS_GSW,SAS,Spurs,GSW,Warriors,H,4,2,SAS,2013-05-06,W_L
124,2011,SAS_MEM,SAS,Spurs,MEM,Grizzlies,H,2,4,MEM,2011-04-17,L_W
125,2014,SAS_MIA,SAS,Spurs,MIA,Heat,H,4,1,SAS,2014-06-05,W_L
126,2012,SAS_OKC,SAS,Spurs,OKC,Thunder,H,2,4,OKC,2012-05-27,W_W


In [212]:
result[result["series"] == "DAL_HOU"]

,year_id,series,team_id,fran_id,opp_id,opp_fran,game_location,num_series_wins,opp_series_wins,series_winner,date_game,game_1_2
30,2005,DAL_HOU,DAL,Mavericks,HOU,Rockets,H,4,3,DAL,2005-04-23,L_L


In [213]:
result[result["series"] == "HOU_DAL"]

,year_id,series,team_id,fran_id,opp_id,opp_fran,game_location,num_series_wins,opp_series_wins,series_winner,date_game,game_1_2
52,2015,HOU_DAL,HOU,Rockets,DAL,Mavericks,H,4,1,HOU,2015-04-18,W_W


### 2. Die erste Auswertung

Ermitteln Sie für die zu untersuchenden Fälle "W_L" und "L_W"  aus Sicht des Auswärtsteams folgende Kennzahlen: 
- Anzahl der Serien
- gewonnene Serien
- verlorene Serien
- Anteil der Siege in %


Das Ergebnis hat zwei Zeilen in denen die vier Kennzahlen und der jeweilige Fall dargestellt werden. 

Was können Sie zum Ergebnis sagen? Hat sich die Vermutung zum Einfluß der Siegreihenfolge bestätigt?


In [214]:
num_series = result.loc[(result["game_1_2"] == "W_L") | (result["game_1_2"] == "L_W"),:].groupby("game_1_2", as_index=False)["game_1_2"].agg({'num_results':'count'})

In [215]:
num_series

,game_1_2,num_results
0,L_W,37
1,W_L,34


In [216]:
#Heim-Team gewinnt die Series
num_home = result.loc[((result["game_1_2"] == "W_L") | (result["game_1_2"] == "L_W")) & (result["team_id"] == result["series_winner"]), :].groupby("game_1_2", as_index=False)["game_1_2"].agg({'home_wins':'count'})

In [217]:
num_home

,game_1_2,home_wins
0,L_W,23
1,W_L,20


In [218]:
#Auswärts-Team gewinnt die Series
num_away = result.loc[((result["game_1_2"] == "W_L") | (result["game_1_2"] == "L_W")) & (result["opp_id"] == result["series_winner"]), :].groupby("game_1_2", as_index=False)["game_1_2"].agg({'away_wins':'count'})

In [219]:
num_away

,game_1_2,away_wins
0,L_W,14
1,W_L,14


In [220]:
stats = pd.merge(left=num_series,right=num_home, on="game_1_2")
stats = pd.merge(left=stats,right=num_away, on="game_1_2")
stats

,game_1_2,num_results,home_wins,away_wins
0,L_W,37,23,14
1,W_L,34,20,14


In [221]:
stats["win_rate"] = stats["away_wins"]/stats["num_results"]
stats

,game_1_2,num_results,home_wins,away_wins,win_rate
0,L_W,37,23,14,0.378378
1,W_L,34,20,14,0.411765


### 3. Eine erweiterte Untersuchung

Erweitern Sie den Bereich der untersuchten Spiele auf alle Best-of-7-Serien Seit 1984 (inkl.). Achtung: Hier wurden teilweise auch Best-Of-3 und Best-Of-5 Serien gespielt. Diese sind hier auszuschließen.

Ermitteln Sie die Kennzahlen zu dem erweiterten Datenbestand analog zur vorigen Aufgabe.

In [222]:
playoffs1984_merged = pd.read_csv("nba_hypo1984.csv")
playoffs1984_filtered = playoffs1984_merged.loc[(playoffs1984_merged["num_series_wins"] ==4)|(playoffs1984_merged["opp_series_wins"] ==4),:]

In [223]:
playoffs1984_filtered = playoffs1984_filtered[playoffs1984_filtered["_iscopy"] == 0]
series1984 = pd.DataFrame(playoffs1984_filtered.groupby(["series", "year_id", "team_id", "fran_id", "opp_id", "opp_fran", "game_location", "num_series_wins", "opp_series_wins", "series_winner"]).date_game.min())
series1984.reset_index(inplace=True)
series1984.set_index(["series","year_id"], inplace=True)
playoffs1984_filtered =  playoffs1984_filtered.loc[(playoffs1984_filtered["series_game"] == 1.0) | (playoffs1984_filtered["series_game"] == 2.0) ,["year_id","series","series_game","game_result"]]
playoffs1984_filtered["game_1_2"] = playoffs1984_filtered.groupby(["year_id","series"])["game_result"].transform(lambda x : '_'.join(x))
playoffs1984_filtered = playoffs1984_filtered.loc[(playoffs1984_filtered["series_game"]==1.0),["year_id", "series", "game_1_2"]]
result1984 = pd.merge(left=series1984,right=playoffs1984_filtered,on=["year_id","series"])

In [226]:
num_series1984 = result1984.loc[(result1984["game_1_2"] == "W_L") | (result1984["game_1_2"] == "L_W"),:].groupby("game_1_2", as_index=False)["game_1_2"].agg({'num_results':'count'})
#Heim-Team gewinnt die Series
num_home1984 = result1984.loc[((result1984["game_1_2"] == "W_L") | (result1984["game_1_2"] == "L_W")) & (result1984["team_id"] == result1984["series_winner"]), :].groupby("game_1_2", as_index=False)["game_1_2"].agg({'home_wins':'count'})
#Auswärts-Team gewinnt die Series
num_away1984 = result1984.loc[((result1984["game_1_2"] == "W_L") | (result1984["game_1_2"] == "L_W")) & (result1984["opp_id"] == result1984["series_winner"]), :].groupby("game_1_2", as_index=False)["game_1_2"].agg({'away_wins':'count'})
stats1984 = pd.merge(left=num_series1984,right=num_home1984, on="game_1_2")
stats1984 = pd.merge(left=stats1984,right=num_away1984, on="game_1_2")
stats1984["win_rate"] = stats1984["away_wins"]/stats1984["num_results"]

In [227]:
stats1984

,game_1_2,num_results,home_wins,away_wins,win_rate
0,L_W,67,38,29,0.432836
1,W_L,67,43,24,0.358209
